In [ ]:
# Importing necessary packages:
from glob import glob
from PipeLine import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
csv_files = glob("./N*/*all*.csv")
geometry='cylinder'

properties_csvs = glob("./N*-properties.csv")

all_properties = []
for properties_csv in properties_csvs:
    df = pd.read_csv(properties_csv,index_col=0)
    all_properties.append(df) 
all_properties = pd.concat(all_properties)
all_properties.reset_index(inplace=True,drop=True)



rhist_name_crd = 'rHistsCrdSegments'
rextensions_crd = ['_rHistsCrd.csv','_rEdgesCrd.csv']
rhist_name_ens_crd = 'rHistsCrd'
rhist_ensembles_crd = PipeLine.simulation_from_segments(csv_files,rhist_name_crd,geometry,rextensions_crd,rhist_name_ens_crd,to_file=False)
rhist_name_mon = 'rHistsMonSegments'
rextensions_mon = ['_rHistsMon.csv','_rEdgesMon.csv']
rhist_name_ens_mon = 'rHistsMon'
rhist_ensembles_mon = PipeLine.simulation_from_segments(csv_files,rhist_name_mon,geometry,rextensions_mon,rhist_name_ens_mon,to_file=False)


direction = 'cyl_radial'
dir_name = 'r'
_, _ = PipeLine.rho_phi_generator_all(rhist_ensembles_crd, all_properties, geometry, direction, dir_name, 'dcrowd','Crd',to_file=True)
_, _ = PipeLine.rho_phi_generator_all(rhist_ensembles_mon, all_properties, geometry, direction, dir_name, 'dmon','Mon',to_file=True)

zhist_name_crd = 'zHistsCrdSegments'
zextensions_crd = ['_zHistsCrd.csv','_zEdgesCrd.csv']
zhist_name_ens_crd = 'zHistsCrd'
zhist_ensembles_crd = PipeLine.simulation_from_segments(csv_files,zhist_name_crd,geometry,zextensions_crd,zhist_name_ens_crd,to_file=False)
zhist_name_mon = 'zHistsMonSegments'
zextensions_mon = ['_zHistsMon.csv','_zEdgesMon.csv']
zhist_name_ens_mon = 'zHistsMon'
zhist_ensembles_mon = PipeLine.simulation_from_segments(csv_files,zhist_name_mon,geometry,zextensions_mon,zhist_name_ens_mon,to_file=False)

thetahist_name_crd = 'thetaHistsCrdSegments'
thetaextensions_crd = ['_thetaHistsCrd.csv','_thetaEdgesCrd.csv']
thetahist_name_ens_crd = 'thetaHistsCrd'
thetahist_ensembles_crd = PipeLine.simulation_from_segments(csv_files,thetahist_name_crd,geometry,thetaextensions_crd,thetahist_name_ens_crd,to_file=False)
thetahist_name_mon = 'zHistsMonSegments'
thetaextensions_mon = ['_thetaHistsMon.csv','_thetaEdgesMon.csv']
thetahist_name_ens_mon = 'thetaHistsMon'
thetahist_ensembles_mon = PipeLine.simulation_from_segments(csv_files,thetahist_name_mon,geometry,thetaextensions_mon,thetahist_name_ens_mon,to_file=False)

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2)
particle_types = ['dcrowd','dmon']
for (ens_name, ens_df), axis in zip(rphi_r_dicts_crd.items(),axes):
    condition = all_properties.filename == ens_name
    dparticle = all_properties[condition]['dcrowd'].to_numpy()[0]
    ens_df['phi_c_norm'] = ens_df.iloc[:,0]/dparticle
    ens_df['phi_c_norm'] = ens_df.loc[:,'phi_c_norm'] / ens_df['phi_c_norm'].sum()
    ens_df['phi_c_norm'].plot(ax=axis,legend=False)
    dparticle = all_properties[condition]['dmon'].to_numpy()[0]
    rphi_r_dicts_mon[ens_name]['phi_m_norm'] = rphi_r_dicts_mon[ens_name].iloc[:,0]/dparticle
    rphi_r_dicts_mon[ens_name]['phi_m_norm'] = rphi_r_dicts_mon[ens_name]['phi_m_norm']/ rphi_r_dicts_mon[ens_name]['phi_m_norm'].sum()
    rphi_r_dicts_mon[ens_name]['phi_m_norm'].plot(ax=axis,legend=False)
    rphi_r_dicts_mon[ens_name]['sum'] = ens_df['phi_c_norm'] + rphi_r_dicts_mon[ens_name]['phi_m_norm']
    rphi_r_dicts_mon[ens_name]['sum'] = rphi_r_dicts_mon[ens_name]['sum'] / rphi_r_dicts_mon[ens_name]['sum'].sum()
    rphi_r_dicts_mon[ens_name]['sum'].plot(ax=axis,legend=False)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))
fontsize = 24
color = 'tab:blue'
line1, = ax.plot(ens_avg_phi_r.iloc[:,0], markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(ens_avg_rho_r.iloc[:,0], rho, markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
import os
for direct in rhist_df.columns:
    path = os.path.join("./", direct)
    os.mkdir(path)

In [ ]:
fpath = hist_files[24]

histo_collections = np.loadtxt(fpath[0],dtype=np.int32)
bin_edges = np.around(np.loadtxt(fpath[1]),decimals=2)
cell_attrs = PipeLine.cellAttributes(fpath[0],geometry='cylinder',printname=True)
filename = cell_attrs.filename
cylindrical_shell_integrand = lambda r: 2 * np.pi * cell_attrs.dcyl * r

# the sum of rho is not equal to the bulk number density (r=infiity) natom/cell_vol
# this arises from the way we descritize the local number desnity.
rho = PipeLine.local_number_density(histo_collections, bin_edges, cylindrical_shell_integrand)

rmon = cell_attrs.dmon / 2.0
bin_centers = np.around((bin_edges[:-1] + bin_edges[1:]) / 2.0,decimals=2)

range_of_bins = PipeLine.bin_edge_index(rmon, bin_centers, bin_edges)
volume_shares = PipeLine.vol_shares(rmon, bin_centers, bin_edges, range_of_bins, PipeLine.sphere_cylinder_intersection)

# the sum of phi is not equal to the bulk volume fraction (r=infiity) natom*vol_per_atom/cell_vol
# this arises from the way we descritize the local volume fraction and the way we relate it to the local number density.
phi = PipeLine.local_volume_fraction(bin_centers, rho, volume_shares)

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))

fontsize = 24
color = 'tab:blue'
line1, = ax.plot(bin_centers, phi, label = r"$\phi_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(bin_centers, rho, label = r"$\rho_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
nens = 0
hists = 0
phi_c = 0.0 

for filename in ens_avg_properties.filename:
    condition = ens_avg_properties['filename']==filename
    for col in rhist_dfs.columns:
        if col.split('ens')[0] == filename.split('_ens_avg')[0]:
            #print(rhist_dfs[col])
            PipeLine.histo_plotter(ax, rhist_dfs[col].values, lambda r: 2 * ens_avg_properties[condition].dcyl / 2.0 * r, rhist_dfs.index.values, label = r"$\phi_c={}$".format(ens_avg_properties[condition].vfrc_crowd), markersize=0.1)
    #ens_evg_rhist
    
#PipeLine.histo_plotter(ax, hists, lambda r: 2 * cell_attrs.dcyl / 2.0 * r, edges, label = r"$\phi_c={}$".format(phi_c), markersize=0.1)
ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")